In [4]:
# 增加site_data
from selenium import webdriver
import requests
import time
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import pymysql
import re
from lxml import html

#取得景點資訊
def getSite(url, n, c):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
                         
    # name
    try:
        name = n
                         
    except AttributeError:
        pass
    
    # city_name
    try:
        c_name = c
                         
    except AttributeError:
        c_name = "None"
                         
    # address
    try:
#         global address
#         for i in soup.select('span.street-address'):
#             address = i.text
        global address
        add =  soup.find('div','attractions-contact-card-ContactCard__contactRow--3Ih6v')
        address = add.get_text().strip()
                         
    except AttributeError:
        address = "None"
                         
    # rate
    try:
        score = soup.find('div', 'ratingContainer')
        rate = score.span['alt']
        Rate = re.findall(r'[\d\.\d]+', rate)
        rate = Rate[0]
                         
    except AttributeError:
        rate = 0
                         
    # comment
    try:
        comment = soup.find('span', 'reviewCount')
        c_comment = comment.get_text()
        Com = re.findall(r'[\d\,\d]+', c_comment)
        c_comment = int(Com[0].replace(",", ""))
                         
    except AttributeError:
        c_comment = 0
                         
    new_id = get_newID()
    href = url
    data_type = "site"
    color = "lightblue"
    shape = "circle"

    # id,name,city,address,type,comment,rate,href,color,shape
    add_Data(new_id, name, c_name, address,
             data_type, c_comment, rate, href, color, shape)
    print("景點名：" + name )
print("="*100)


def get_newID():
    last_id = ""
    resta_id = "SELECT MAX(id) FROM site_data"
    cur.execute(resta_id)
    for id in cur:
        num = re.findall(r'[\d\.\d]+', id[0])
        num = int(num[0]) + 1
        last_id = 'S{0:04}'.format(num)
    return last_id

#加入site資料到資料庫
def add_Data(id, name, city_name, address, type, c_cmt, rate, href, color, shape):
    add_data = ("INSERT INTO site_data"
                "(id,name,city_name,address,type,comment,rate,href,color,shape) "
                "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)")
    data = (id, name, city_name, address, type, c_cmt, rate, href, color, shape)
    cur.execute(add_data, data)
    # cnx.commit()


#Main
tStart = time.time()

cnx = pymysql.connect(
    host="localhost",
    user="root",
    passwd="12345",
    database='test'
)

cur = cnx.cursor()
cursor = cnx.cursor()
ch_cur = cnx.cursor()
u_cur = cnx.cursor()

# url = "https://www.tripadvisor.com.tw/Restaurant_Review-g13806900-d4600419-Reviews-Sunny_Hills-Songshan_Taipei.html"
hr = "SELECT near_href,near_site, city_name FROM comment_relationship"
cursor.execute(hr)

for j in cursor.fetchall():
    # near
    url = j[0]
    name = j[1]
    city = j[2]
    check = url[31:41]
    
    if check == "Attraction":
        a_check = "SELECT id FROM site_data WHERE name = '" + name + "' and city_name = '" + city + "'"
        ch_cur.execute(a_check)
        entry = ch_cur.fetchone()
        if entry is None:
            getSite(url, name, city)
                
tEnd = time.time()
print("Run Time：", tEnd - tStart)

Run Time： 3.143162727355957
